In [1]:
import codecs
import copy
import json

from collections import OrderedDict

In [2]:
# generate user configs for handycat qe score experiments

experiment_config_tsv = '../app/data/qe_score_experiments/Handycat_QE_score_random_subjects_grid.csv'

In [3]:
with codecs.open(experiment_config_tsv, encoding='utf8') as conf:
    config_rows = [r.split(',') for r in conf.read().strip().split('\r\n')]

column_headers = config_rows[0]
user_configs = config_rows[1:]
user_configs = [[r[0]] + [int(s) for s in r[1:]] for r in user_configs]

print(column_headers)

# pair scores with docs ids 
user_configs = [[r[0]] + zip(r[1:5], r[5:9]) for r in user_configs]

[u'user_id', u'doc_id_0', u'doc_id_1', u'doc_id_2', u'doc_id_3', u'score_0', u'score_1', u'score_2', u'score_3']


In [4]:
len(user_configs)

20

In [5]:
project_names = ['PRJ0', 'PRJ1', 'PRJ2', 'PRJ3']

In [6]:
# base urls
# data/qe_score_experiments/documents/PRJ0.xliff
# data/qe_score_experiments/documents/score_tsvs/QE0

In [7]:
# translator_group_base_obj = {
#     'name': '',
#     'sampleFiles': [],   
# }

# project_config_base_obj = {
#     'name'         : '',
#     'internalName' : '',
#     'url'          : '',
#     'configuration': {
#       'target': {
#         'activeComponent' : 'typeaheadEditor',
#         'defaultComponent': 'typeaheadEditor',
#         'components'      : [
#           {
#             'directiveName': 'typeaheadEditor',
#             'textName'     : 'Typeahead Editor'
#           }
#         ]
#       },
#       'tsvUrl': '',
#       'tsvData': [],
#       'qeScoreConfig': {
#         'scoreIndex': ''
#       }
#     } 
# }

In [8]:
translator_group_base_obj = OrderedDict([('name', ''), ('sampleFiles', [])])

target_config_base_obj = OrderedDict([
    ('activeComponent', 'typeaheadEditor'),
    ('defaultComponent', 'typeaheadEditor'),
    ('components', OrderedDict([
            ('directiveName', 'typeaheadEditor'),
            ('textName', 'typeaheadEditor')
        ])
    )
])

internal_config_base_obj = OrderedDict([
    ('target', target_config_base_obj),
    ('tsvUrl', ''),
    ('tsvData', []),
    ('qeScoreConfig', {'scoreIndex': ''})
])


project_config_base_obj = OrderedDict([('name', ''),
                                      ('internalName', ''),
                                      ('url', ''),
                                      ('configuration', internal_config_base_obj)])

In [9]:
# use the config templates, replicate as many times as necessary to generate the full config object

all_groups = []
for user_config in user_configs:
    user_id = user_config[0]
    project_configs = user_config[1:]
    
    user_obj = copy.deepcopy(translator_group_base_obj)
    user_obj['name'] = user_id
    
    for idx, (project_id, score_id) in enumerate(project_configs):
        project_config = copy.deepcopy(project_config_base_obj)
        project_name = project_names[project_id]
        project_config['name'] = u'{}: task: {} project name: {}'.format(user_id, idx, project_name)
        project_config['internal_name'] = project_name
        project_config['url'] = 'data/qe_score_experiments/documents/PRJ{}.xliff'.format(project_id)
        project_config['configuration']['tsvUrl'] = 'data/qe_score_experiments/documents/score_tsvs/QE{}'.format(score_id)
        project_config['configuration']['qeScoreConfig']['scoreIndex'] = score_id
        
        user_obj['sampleFiles'].append(project_config)
    
    all_groups.append(user_obj)


In [10]:
print(json.dumps(all_groups, indent=2))

[
  {
    "name": "Translator 1", 
    "sampleFiles": [
      {
        "name": "Translator 1: task: 0 project name: PRJ0", 
        "internalName": "", 
        "url": "data/qe_score_experiments/documents/PRJ0.xliff", 
        "configuration": {
          "target": {
            "activeComponent": "typeaheadEditor", 
            "defaultComponent": "typeaheadEditor", 
            "components": {
              "directiveName": "typeaheadEditor", 
              "textName": "typeaheadEditor"
            }
          }, 
          "tsvUrl": "data/qe_score_experiments/documents/score_tsvs/QE0", 
          "tsvData": [], 
          "qeScoreConfig": {
            "scoreIndex": 0
          }
        }, 
        "internal_name": "PRJ0"
      }, 
      {
        "name": "Translator 1: task: 1 project name: PRJ1", 
        "internalName": "", 
        "url": "data/qe_score_experiments/documents/PRJ1.xliff", 
        "configuration": {
          "target": {
            "activeComponent": "typeahea

In [12]:
config_output = 'qe_score_experiment_config.json'

with codecs.open(config_output, 'w', 'utf8') as out:
    out.write(json.dumps(all_groups, indent=2))